# Install Dependencies

In [ ]:
!pip install pgvector sqlalchemy psycopg2-binary jb-manager-bot tqdm python-dotenv json-repair

# Setup Environment

In [1]:
from crud import session

from pgvector.sqlalchemy import Vector
from sqlalchemy.orm import declarative_base, mapped_column
from sqlalchemy import ARRAY, Column, String, select
from sqlalchemy.dialects.postgresql import JSON

Base = declarative_base()


class GrievanceCategory(Base):
    __tablename__ = "grievance_category_v2"
    id = Column(String, primary_key=True)
    ministry = Column(String)
    category = Column(String)
    subcategory = Column(ARRAY(String))
    description = Column(String)
    embedding = mapped_column(Vector(1536))
    fields = Column(ARRAY(JSON))

    def __repr__(self):
        return f"<GrievanceCategory(ministry={self.ministry}, category={self.category}, subcategory={self.subcategory}>"

# Helper functions for processing messages

In [2]:
from datetime import datetime
from pprint import pprint
from openai_utils import sm, um, am, llm, generate_embeddings

#### Generate Summary

In [3]:
conversation_history = []

system_prompt = f"You are a helpful assistant, the CPGRAMS bot, working with the Prime Minister of India’s office. Your role is to help users share their grievances on a grievance redressal platform for the Government of India. You will not offer any solutions for their grievance. You will merely try to extract as much information as possible to allow a grievance officer to deal with the grievance effectively. Current date: {datetime.now().date().isoformat()}"
system_prompt = sm(system_prompt)
conversation_history.append(system_prompt)
system_prompt

{'role': 'system',
 'content': 'You are a helpful assistant, the CPGRAMS bot, working with the Prime Minister of India’s office. Your role is to help users share their grievances on a grievance redressal platform for the Government of India. You will not offer any solutions for their grievance. You will merely try to extract as much information as possible to allow a grievance officer to deal with the grievance effectively. Current date: 2024-04-25'}

In [4]:
start_message = "Hi, How can I help you?"
start_message = am(start_message)
conversation_history.append(start_message)
start_message

{'role': 'assistant', 'content': 'Hi, How can I help you?'}

In [5]:
user_input = "I am unable to access my PAN card from digilocker"
user_input = um(user_input)
conversation_history.append(user_input)
user_input

{'role': 'user',
 'content': 'I am unable to access my PAN card from digilocker'}

In [6]:
system_prompt = f"""
                Analyze the above conversation in detail and give your analysis. 
                Your analysis should be detailed enough such that it could be used to determine the grievance category from a database.
                Do not try to determine the grievance category in this step.
                Current date and time: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
                Process any date and time related information in the conversation.
                """
print(system_prompt)


                Analyze the above conversation in detail and give your analysis. 
                Your analysis should be detailed enough such that it could be used to determine the grievance category from a database.
                Do not try to determine the grievance category in this step.
                Current date and time: 2024-04-25 15:16:17
                Process any date and time related information in the conversation.
                


In [7]:
llm_system_prompt = sm(system_prompt)
messages = conversation_history + [llm_system_prompt]
pprint(messages)

[{'content': 'You are a helpful assistant, the CPGRAMS bot, working with the '
             'Prime Minister of India’s office. Your role is to help users '
             'share their grievances on a grievance redressal platform for the '
             'Government of India. You will not offer any solutions for their '
             'grievance. You will merely try to extract as much information as '
             'possible to allow a grievance officer to deal with the grievance '
             'effectively. Current date: 2024-04-25',
  'role': 'system'},
 {'content': 'Hi, How can I help you?', 'role': 'assistant'},
 {'content': 'I am unable to access my PAN card from digilocker',
  'role': 'user'},
 {'content': '\n'
             '                Analyze the above conversation in detail and '
             'give your analysis. \n'
             '                Your analysis should be detailed enough such '
             'that it could be used to determine the grievance category from a '
        

In [8]:
summary, cost = llm(messages, model="gpt-4-turbo-preview")
print(summary)

The user's grievance revolves around an issue with accessing their Permanent Account Number (PAN) card from DigiLocker. DigiLocker is a digital platform launched by the Government of India aimed at ensuring that citizens have access to authentic digital documents in a digital wallet provided by the government. The user's inability to access their PAN card from this platform can be attributed to several potential causes, such as technical glitches within the DigiLocker system, issues related to the linking of the PAN with the user's DigiLocker account, or discrepancies in the user's personal information that may be causing a mismatch.

Key elements to extract and analyze for effectively categorizing and addressing this grievance include:

1. **Nature of the Issue**: The problem is specifically related to accessing a government-issued document (PAN card) through a government-provided digital service (DigiLocker). This indicates that the grievance falls within the domain of digital govern

```generate_summary``` - Function to generate summary of the user conversation.

1. ```conv_history``` - Conversation history including initial system prompt. Function will internally handle the system prompts.

In [9]:
def generate_summary(conv_history):
    conv_summary, cost = llm(
        messages=conv_history
        + [
            sm(
                f"""
                Analyze the above conversation in detail and give your analysis. 
                Your analysis should be detailed enough such that it could be used to determine the grievance category from a database.
                Do not try to determine the grievance category in this step.
                Current date and time: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
                Process any date and time related information in the conversation.
                """
            )
        ],
        model="gpt-4-turbo-preview",
    )
    return conv_summary, cost

In [10]:
summary, _ = generate_summary(conversation_history)
print(summary)

The user's grievance revolves around an issue with accessing their Permanent Account Number (PAN) card from DigiLocker. DigiLocker is a digital platform provided by the Government of India for the storage, sharing, and verification of documents and certificates. The user's inability to access their PAN card through this platform can be attributed to several potential causes, such as technical glitches, user account issues, or discrepancies in linking the PAN with the DigiLocker account.

Key elements to extract and analyze for effectively categorizing and addressing this grievance include:

1. **Nature of the Issue**: The problem is specifically related to accessing a PAN card on DigiLocker, which indicates it falls under issues related to digital government services, specifically those involving document access and management.

2. **Possible Causes**:
   - Technical issues with the DigiLocker platform, such as server downtime or bugs affecting document retrieval.
   - User account pro

#### Get Relevant Grievance Category

In [11]:
summary_embeddings, _ = generate_embeddings(conversation_summary=[summary])
summary_embedding = summary_embeddings[0]

In [12]:
len(summary_embedding)

1536

In [13]:
top_k = 10
relevant_categories = session.scalars(
    select(GrievanceCategory)
    .order_by(GrievanceCategory.embedding.l2_distance(summary_embedding))
    .limit(top_k)
).all()
relevant_categories

[<GrievanceCategory(ministry=Unique Identification Authority of India, category=Linking of Aadhaar related issues, subcategory=['Linking of Aadhaar with PAN']>,
 <GrievanceCategory(ministry=Central Board of Direct Taxes (Income Tax), category=PAN Issues, subcategory=['Other']>,
 <GrievanceCategory(ministry=Central Board of Direct Taxes (Income Tax), category=PAN Issues, subcategory=['Delay in PAN issues']>,
 <GrievanceCategory(ministry=Central Board of Direct Taxes (Income Tax), category=PAN Issues, subcategory=['Wrong PAN number allotted']>,
 <GrievanceCategory(ministry=Central Board of Direct Taxes (Income Tax), category=PAN Issues, subcategory=['Mistakes in PAN card']>,
 <GrievanceCategory(ministry=Central Board of Direct Taxes (Income Tax), category=PAN Issues, subcategory=['Same PAN numbers allotted to multiple users']>,
 <GrievanceCategory(ministry=Ministry of Electronics & Information Technology, category=Digital Services (CSC/MyGov/NeGD/NIC), subcategory=['NeGD', 'DigiLocker']>

```get_closest_grievance_category``` - Function will fetch most relevant grievance bucket from the vectorDB. This is based on the query provided.


1. ```conversation_summary``` - Summary of the conversation by the user.
2. ```top_k``` - (Default = 5) Number of buckets need to be fetched.

In [14]:
import json


def get_closest_grievance_category(conversation_summary: str, top_k: int = 5):
    summary_embeddings, cost = generate_embeddings(
        conversation_summary=[conversation_summary]
    )
    summary_embedding = summary_embeddings[0]
    result = session.scalars(
        select(GrievanceCategory)
        .order_by(GrievanceCategory.embedding.l2_distance(summary_embedding))
        .limit(top_k)
    ).all()

    option_template = "Available grievance categories \n {options}"
    options = ""
    fields = []
    buckets = []
    for index, grievance in enumerate(result):
        options = (
            options
            + f"\n{index+1}. Bucket Id: {index}\nMinistry: {grievance.ministry}\nCategory: {grievance.category}\nSub Category: {grievance.subcategory}\nDescription: {grievance.description}"
        )
        buckets.append(grievance)
        fields.append(grievance.fields)
    closest_buckets = option_template.format(options=options)

    return closest_buckets, fields, buckets, cost

In [15]:
possible_buckets_text, fields, buckets_object, _ = get_closest_grievance_category(
    summary, top_k=10
)

In [16]:
for bucket in buckets_object:
    print(bucket)

<GrievanceCategory(ministry=Unique Identification Authority of India, category=Linking of Aadhaar related issues, subcategory=['Linking of Aadhaar with PAN']>
<GrievanceCategory(ministry=Central Board of Direct Taxes (Income Tax), category=PAN Issues, subcategory=['Other']>
<GrievanceCategory(ministry=Central Board of Direct Taxes (Income Tax), category=PAN Issues, subcategory=['Delay in PAN issues']>
<GrievanceCategory(ministry=Central Board of Direct Taxes (Income Tax), category=PAN Issues, subcategory=['Wrong PAN number allotted']>
<GrievanceCategory(ministry=Central Board of Direct Taxes (Income Tax), category=PAN Issues, subcategory=['Mistakes in PAN card']>
<GrievanceCategory(ministry=Central Board of Direct Taxes (Income Tax), category=PAN Issues, subcategory=['Same PAN numbers allotted to multiple users']>
<GrievanceCategory(ministry=Ministry of Electronics & Information Technology, category=Digital Services (CSC/MyGov/NeGD/NIC), subcategory=['NeGD', 'DigiLocker']>
<GrievanceCa

In [17]:
print(possible_buckets_text)

Available grievance categories 
 
1. Bucket Id: 0
Ministry: Unique Identification Authority of India
Category: Linking of Aadhaar related issues
Sub Category: ['Linking of Aadhaar with PAN']
Description: This category of grievance pertains to issues related to the linking of Aadhaar with the Permanent Account Number (PAN) in India. Citizens may encounter problems or face difficulties while attempting to link their Aadhaar number with their PAN card. This could include technical issues on the income tax department's portal, errors in the data provided, or failure to link due to mismatch of details.

Some possible ways in which people may refer to this grievance include:
- Linking of Aadhaar with PAN card
- Aadhaar-PAN linking issues
- Problems in linking Aadhaar and PAN
- Unable to link Aadhaar and PAN
- Errors in Aadhaar-PAN linking

It is important for individuals to ensure that their Aadhaar and PAN details are linked correctly as it is a mandatory requirement for various financial a

```determine_grievance_bucket_and_followup_questions``` - Function to generate follow up questions as well as grievance buckets(if possible) in which the user query can belong. 

1. ```conversation_history``` - conversation history which will be analyzed to generate the followup questions and determine buckets.
2. ```possible_buckets``` - List of possible choices of the grievance buckets. Generally the output from vectorDB call can suffice after incorporating test formatting.

In [18]:
def determine_grievance_bucket_and_followup_questions(
    conversation_history, possible_buckets
):
    system_prompt = """You are a helpful assistant, the CPGRAMS bot, working with the Prime Minister of India’s office. Your role is to help users share their grievances on a grievance redressal platform for the Government of India.
    Here is a set of possible grievance buckets in which the user could raise a grievances. 
    Possible grievance buckets:
    {options}
    
    Analyze the conversation history and the user input. 
    Your primary task is to determine most relevant grievance bucket in which the user query can fall.
    Do not recommend grievance buckets if you do not have enough required information or you are not sure about. 
    Do not recommend grievance buckets that do not satisfy all the categories and subcatgories according to the information provided by user.
    If you think you do not have enough information, generate a short follow up question that can be asked to the user whose response can help in determining the grievance bucket.
    Also, generate an brief empathetic prefix message to show to the user before asking the followup question.
    Do not recommend grievance buckets if multiple grievance buckets from the list satisfing the criteria given by required information. Ask the user to provide more information.
    The follow up question should be very diverse. 
    You can ask upto 1 follow up question, not more than 1. 
    The follow up question should be very basic, do not ask question that you think a 15 year old would not be able to answer.
    You will not offer any solutions for their grievance. You will merely try to extract as much information to exactly determine the grievance bucket.
    Since you are not offering any solutions, you can ask any question that you think can help you determine the grievance bucket.
    Analyze and tell why you think the grievance falls under the grievance bucket. If you don't think any grievance bucket from the provided list is relevant, tell the reason for it.


    Give the output in the following json format:
    {{
        "grievance_bucket": 
            {{
                "bucket_id": "bucket_id",
                "ministry": "ministry",
                "category": "category",
                "subcategory": ["subcategory"],
                "description": "description"
            }}
        ,
        "follow_up_questions": [
            "question 1"
        ],
        "user_message": "Empathetic prefix message to show to user in a conversational manner. The message should be short and include emojis to show empathy and make the user feel comfortable."
    }}
    Provide empty list if no follow up questions are mentioned.
    Provide none if no grievance buckets are mentioned by user. Do not generate new grievance buckets. Only consider the grievance buckets provided.
    """
    system_prompt = sm(system_prompt.format(options=possible_buckets))
    buckets_and_questions, cost = llm(
        messages=conversation_history + [system_prompt],
        model="gpt-4-turbo-preview",
        response_format={"type": "json_object"},
    )
    response = json.loads(buckets_and_questions)
    return response, cost

In [19]:
grievance_buckets, _ = determine_grievance_bucket_and_followup_questions(
    conversation_history, possible_buckets_text
)

In [20]:
pprint(grievance_buckets)

{'follow_up_questions': [],
 'grievance_bucket': {'bucket_id': '6',
                      'category': 'Digital Services (CSC/MyGov/NeGD/NIC)',
                      'description': 'The DigiLocker service provided by the '
                                     'National e-Governance Division (NeGD) '
                                     'under the Ministry of Electronics & '
                                     'Information Technology allows users to '
                                     'store, access, and share digital copies '
                                     'of important documents and '
                                     'certificates. \n'
                                     '\n'
                                     'Users may have the following grievances '
                                     'related to DigiLocker:\n'
                                     '1. Difficulty in accessing or signing up '
                                     'for DigiLocker account\n'
           

```generate_field_questions``` - Function to generate follow up questions specific to the fields which is required to file a grievance. It will analyze the information and generate the questions accordingly.  

1. ```grievance_bucket``` - Grievance bucket which is predicted in previous step.
2. ```required_fields``` - List of fields required to filled by user to file the grievance.
3. ```conversation_summary``` - Summary of the conversation.

In [21]:
def generate_field_questions(grievance_bucket, required_fields, conversation_summary):
    system_prompt = """
    You are a helpful assistant, the CPGRAMS bot, working with the Prime Minister of India’s office. Your role is to help users share their grievances on a grievance redressal platform for the Government of India.
    
    You'll be grievance bucket, containing ministry name, category, sub categories and description of grievance in which user can file grievance.
    You're also provided the fields required to file the grievance.
    You'll be provided the conversation summary of the user.
    Analyze the conversation summary and determine the value of fields. 
    Some fields might require description of the grievance, so infer the description on your own instead of asking the user.
    Analyze and determine if value of all the mandatory fields are present. 
    Analyze the grievance bucket, and generate an brief empathetic prefix message to show to the user before asking the followup question.
    Everything should be brief. 
    If value of all mandatory fields are present, set the value of mandatory_data_present to True, else set it to False.
    If value of all mandatory fields are present, you can tell user in the brief emphatic message that all the required fields are present and ask them to confirm the values of the fields to proceed filing the grievance. 
    Do not generate more than 1 follow up question.
    Your task is to restructure the follow up in json format as shown below:
    {{
        "grievance_bucket": 
            {{
                "bucket_id": "bucket_id",
                "ministry": "ministry",
                "category": "category",
                "subcategory": ["subcategory"],
                "description": "description"
            }}
        ,
        "follow_up_questions": [
            "question 1"
        ],
        "user_message": "Empathetic message to show to user in a conversational manner. The message should be short and include emojis to show empathy and make the user feel comfortable."
        "field_data": 
            {{
                "field_1": "field_1_value", # field_1 is the name of field and field_1_value is the value. use none if the value is not present.
                "field_2": "field_2_value"
            }}
        ,
        "mandatory_data_present": True/False # Set it to True if value for all the mandatory fields are present, else set it to False.
    }}

    Provide empty list if no follow up questions are needed.
    
    """
    user_message = f"""
    Grievance bucket:
    {grievance_bucket}

    Required fields:
    {required_fields}

    Conversation Summary:
    {conversation_summary}
    """
    response, cost = llm(
        messages=[sm(system_prompt), um(user_message)],
        model="gpt-4-turbo-preview",
        response_format={"type": "json_object"},
    )
    response = json.loads(response)
    return response, cost

In [22]:
bucket = grievance_buckets["grievance_bucket"]
bucket

{'bucket_id': '6',
 'ministry': 'Ministry of Electronics & Information Technology',
 'category': 'Digital Services (CSC/MyGov/NeGD/NIC)',
 'subcategory': ['NeGD', 'DigiLocker'],
 'description': 'The DigiLocker service provided by the National e-Governance Division (NeGD) under the Ministry of Electronics & Information Technology allows users to store, access, and share digital copies of important documents and certificates. \n\nUsers may have the following grievances related to DigiLocker:\n1. Difficulty in accessing or signing up for DigiLocker account\n2. Issues with uploading or downloading documents on DigiLocker\n3. Concerns about the security and privacy of personal information stored on DigiLocker\n4. Problems with linking DigiLocker to other government services or platforms\n5. Unauthorized access to DigiLocker account\n6. Technical glitches or errors while using DigiLocker\n7. Unclear guidelines or instructions on how to use DigiLocker effectively\n\nAlternate terms for DigiLo

In [23]:
user_choice = {
    "id": bucket["bucket_id"],
    "title": f"Ministry: {bucket['ministry']} :: Category: {bucket['category']} :: SubCategory: {bucket['subcategory']} :: Description: {bucket['description']}",
}
required_fields = fields[int(user_choice["id"])]
required_fields

[{'field_name': 'Issue_Details',
  'data_type': 'string',
  'mandatory': True,
  'description': 'Please provide details about the issue you are facing related to DigiLocker.'},
 {'field_name': 'Document_Type',
  'data_type': 'string',
  'mandatory': True,
  'description': 'What type of document were you trying to upload/download/access in DigiLocker?'},
 {'field_name': 'Error_Message',
  'data_type': 'string',
  'mandatory': False,
  'description': 'If you received any error message, please provide the exact text.'},
 {'field_name': 'Device_Used',
  'data_type': 'string',
  'mandatory': False,
  'description': 'Which device (e.g., computer, mobile phone) were you using when you encountered the issue?'},
 {'field_name': 'Internet_Connection',
  'data_type': 'boolean',
  'mandatory': False,
  'description': 'Did you have a stable internet connection at the time of the issue?'}]

In [24]:
field_questions = generate_field_questions(user_choice, required_fields, summary)
pprint(field_questions)

({'field_data': {'Device_Used': 'none',
                 'Document_Type': 'PAN card',
                 'Error_Message': 'none',
                 'Internet_Connection': 'none',
                 'Issue_Details': 'Inability to access PAN card on DigiLocker, '
                                  'potentially due to technical issues, '
                                  'account problems, or discrepancies in '
                                  'linking PAN with DigiLocker account.'},
  'follow_up_questions': [],
  'grievance_bucket': {'bucket_id': '6',
                       'category': 'Digital Services (CSC/MyGov/NeGD/NIC)',
                       'description': 'The user is experiencing difficulty '
                                      'accessing their Permanent Account '
                                      'Number (PAN) card on DigiLocker. This '
                                      'issue may be due to technical glitches, '
                                      'user account problems,

`process_message` - Function to process user conversation

1. `conversation_history` - Conversation history


Steps Involved - 
1. Summarise the end to end user conversation and generate a detailed description of user requirements and possible scenarios.
2. Fetch 20 relevant grievance buckets corresponding to detailed description.
3. Generate top 3 relevance buckets from the top 20 relevant buckets by analyzing the user conversation using GPT-4.
4. Generate followup questions to gain more information by analyzing the user conversation using GPT-4.
5. Structure the relevance buckets and followup questions into json format.

In [25]:
def process_message(conversation_history):
    total_cost = 0.0
    conversation_summary, cost = generate_summary(conversation_history)
    total_cost += cost
    possible_buckets, fields, buckets_object, cost = get_closest_grievance_category(
        conversation_summary, top_k=30
    )
    total_cost += cost
    grievance_bucket_followup_questions, cost = determine_grievance_bucket_and_followup_questions(
        conversation_history, possible_buckets
    )
    total_cost += cost

    user_choice = grievance_bucket_followup_questions["grievance_bucket"]
    if user_choice is None:
        return (
            grievance_bucket_followup_questions,
            user_choice,
            grievance_bucket_followup_questions,
            False,
            cost
        )

    user_choice = {
        "id": user_choice["bucket_id"],
        "title": f"Ministry: {user_choice['ministry']} :: Category: {user_choice['category']} :: SubCategory: {user_choice['subcategory']} :: Description: {user_choice['description']}",
    }

    required_fields = fields[int(user_choice["id"])]
    field_questions, cost = generate_field_questions(
        user_choice, required_fields, conversation_summary
    )
    total_cost += cost

    return (
        grievance_bucket_followup_questions,
        user_choice,
        field_questions,
        field_questions["mandatory_data_present"],
        cost
    )

In [26]:
_, _, response, is_mandatory_fields_present, _ = process_message(conversation_history)

print(f"Mandatory Fields Present: {is_mandatory_fields_present}")
pprint(response)

Mandatory Fields Present: True
{'field_data': {'Device_Used': 'none',
                'Document_Type': 'Permanent Account Number (PAN) card',
                'Error_Message': 'none',
                'Internet_Connection': 'none',
                'Issue_Details': 'The user is unable to access their Permanent '
                                 'Account Number (PAN) card from DigiLocker '
                                 'due to potential technical issues, account '
                                 'verification problems, PAN card uploading or '
                                 'linking issues, user authentication '
                                 'failures, or misunderstandings about the '
                                 'process.'},
 'follow_up_questions': [],
 'grievance_bucket': {'bucket_id': '2',
                      'category': 'Digital Services (CSC/MyGov/NeGD/NIC)',
                      'description': 'The DigiLocker service provided by the '
                                  

# Example Conversation

### Example 1

In [27]:
conversation_history = []

system_prompt = sm(
    f"You are a helpful assistant, the CPGRAMS bot, working with the Prime Minister of India’s office. Your role is to help users share their grievances on a grievance redressal platform for the Government of India. You will not offer any solutions for their grievance. You will merely try to extract as much information as possible to allow a grievance officer to deal with the grievance effectively. Current date: {datetime.now().date().isoformat()}"
)
conversation_history.append(system_prompt)

start_message = "Hi, How can I help you?"
print(f"BOT:> {start_message}")

start_message = am(start_message)
conversation_history.append(start_message)

BOT:> Hi, How can I help you?


In [28]:
user_input = "The facilities at government hospital are not good."

print(f"USER:> {user_input}")
print()

user_input = um(user_input)
conversation_history.append(user_input)

_, _, response, is_mandatory_fields_present, cost = process_message(conversation_history)

conversation_history.append(am(response["user_message"]))
print(f'BOT:> {response["user_message"]}')
for question in response["follow_up_questions"]:
    conversation_history.append(am(question))
    print(f"BOT:> {question}")

if is_mandatory_fields_present:
    for field_data in response["field_data"]:
        print(f'{field_data}: {response["field_data"][field_data]}')

USER:> The facilities at government hospital are not good.

BOT:> I'm really sorry to hear that you're facing issues with the facilities at a government hospital 😟. To help you better, could you provide a bit more detail?
BOT:> Could you please specify if your concern is related to the quality of healthcare services, availability of medical supplies, infrastructure, or something else at the government hospital?


In [29]:
print(f"Cost: ${cost}")

Cost: $0.07043


In [30]:
user_input = "The staff was rude to me"
print(f"USER:> {user_input}")
print()

user_input = um(user_input)
conversation_history.append(user_input)

_, _, response, is_mandatory_fields_present, cost = process_message(conversation_history)

conversation_history.append(am(response["user_message"]))
print(f'BOT:> {response["user_message"]}')
for question in response["follow_up_questions"]:
    conversation_history.append(am(question))
    print(f"BOT:> {question}")

if is_mandatory_fields_present:
    for field_data in response["field_data"]:
        print(f'{field_data}: {response["field_data"][field_data]}')

USER:> The staff was rude to me

BOT:> I'm truly sorry to hear about your experience 😟. It's important for us to ensure everyone receives respectful treatment. To assist you better, could you share a bit more detail?
BOT:> Could you please specify which hospital you're referring to? This will help us understand and categorize your grievance accurately.


In [31]:
print(f"Cost: ${cost}")

Cost: $0.06911


In [32]:
user_input = "I went to AIIMS Patna today and nurse told me to get lost when I asked him about my reports."
print(f"USER:> {user_input}")
print()

user_input = um(user_input)
conversation_history.append(user_input)

_, _, response, is_mandatory_fields_present, cost = process_message(conversation_history)

conversation_history.append(am(response["user_message"]))
print(f'BOT:> {response["user_message"]}')
for question in response["follow_up_questions"]:
    conversation_history.append(am(question))
    print(f"BOT:> {question}")

if is_mandatory_fields_present:
    for field_data in response["field_data"]:
        print(f'{field_data}: {response["field_data"][field_data]}')

USER:> I went to AIIMS Patna today and nurse told me to get lost when I asked him about my reports.

BOT:> We're truly sorry to hear about your experience 😔. It seems we have all the necessary details to file your grievance. Could you please confirm the information below to proceed?
date_of_incident: 2024-04-25
department_involved: Nursing
description_of_incident: The user approached a nurse to inquire about their medical reports, and the nurse responded rudely by telling the user to 'get lost.'
witnesses: none
action_taken_previously: none


In [33]:
print(f"Cost: ${cost}")

Cost: $0.015390000000000001
